# PERSONAL PROJECT

In this notebook, I will investigate the data available to us by the Open Data initiative of the French goverment.  The focus will be on house prices within France, and more specifically within Paris.  

Data is open source and is not bound by any commercial or usage constraints.  The data will be downloaded through the site, on this link: https://www.data.gouv.fr/fr/datasets/demandes-de-valeurs-foncieres/

The exploratory visualisations will be available on https://public.tableau.com/profile/rodrigo.baptista





## Importing libraries


In [ ]:
#Set up
import sys
from pathlib import Path
import sklearn
import chardet
import os, shutil


#webscraping
from lxml import html, etree
import requests
import urllib

#data science
import pandas as pd
import numpy as np
import seaborn as sns
import dtale

#visualisation
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np

In [ ]:
# check packages versions
print("\n# check packages versions\n")

print("python: " + sys.version)

print("sklearn: " + sklearn.__version__)

## Useful Functions

In [ ]:
def check_filePath (filePath_varString, filePath):
    """
    Checks if the file exists and prints the value, else exits.
    """
    if not filePath.exists():
        print("ERROR, " + filePath_varString + " not found!!!")
        sys.exit()
    print(filePath_varString + ": " + filePath._str)

In [ ]:
def download(url: str, dest_folder: str, filename: str):
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)  # create folder if it does not exist

    #filename = url.split('/')[-1].replace(" ", "_")  # be careful with file names
    file_path = os.path.join(dest_folder, filename)

    r = requests.get(url, stream=True)
    if r.ok:
        print("saving to", os.path.abspath(file_path))
        with open(file_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024 * 8):
                if chunk:
                    f.write(chunk)
                    f.flush()
                    os.fsync(f.fileno())
    else:  # HTTP status code 4XX/5XX
        print("Download failed: status code {}\n{}".format(r.status_code, r.text))

In [ ]:
def detect_encoding (filePath):
    """
    Returns the encoding of the file
    """
    with open(filePath, mode='rb') as f:
        # result = chardet.detect(f.read(100000))
        detector = chardet.universaldetector.UniversalDetector()
        count = 0
        for line in f.readlines():
            detector.feed(line)
            count += 1
            if count == 1000: break
            if detector.done: break
        detector.close()
        # print(detector.result)\n",
    # print(filePath._str, "\n Encoding: ", result['encoding'])

    # return result['encoding']
    return detector.result['encoding']

## Set up directories

In [ ]:
# Set Data Folder Paths depending on User
print("\n# Set Data Folder Paths for project\n")

# Set Workdir folder where projects are developed
workdir_folder = Path("/workspaces/hsbc-gpmo-account")
if not workdir_folder.exists():
    workdir_folder = Path("C:\\Users\\rodri\\OneDrive\\Documents\\Data Science Projects - Personal\\opendata-france\\House Prices Paris - Datasources")
check_filePath("workdir_folder", workdir_folder)


# Set Source folders where data sources for project are stored
sources_folder = Path("/workspaces/hsbc-gpmo-account")
if not sources_folder.exists():
    sources_folder = Path("C:\\Users\\rodri\\OneDrive\\Documents\\Data Science Projects - Personal\\opendata-france\\House Prices Paris - Datasources")
check_filePath("sources_folder", sources_folder)

## Scraping data

To keep this notebook up to date and relevant, the data download will be automated to fetch directly from the site. This ensures I'm able to have access to any new datasets that are updated to the website.


Looking through the datasets, their names are structured and consistent.  At the time of writing this, the naming convention follows:

    valeursfoncieres-year-extra

The extra component is included for cases where the dataset does not cover a full year.


In [ ]:
# Get the original webpage html content
webpageLink = 'https://www.data.gouv.fr/fr/datasets/demandes-de-valeurs-foncieres/'
page = requests.get(webpageLink)
# convert the data received into searchable HTML
extractedHtml = html.fromstring(page.content)
# use an XPath query to find the datasets link 
# to do this, we search for all download button links and get datasetlink
dataSrc = extractedHtml.xpath('//div[@class="resource-card-actions btn-toolbar"]/a/@href')  
# I'll assume the documentation for the data set will be limited to the 4 links already located at the bottom of the website plus 4 download links for misc resources. So, the relevant files to keep are all but the last 8
dataSrc
dataSrc2=dataSrc[:-8]
dataSrc2.reverse()

Load and concatenate data


In [ ]:
#It's now ready to download the files and combine them
# we will only download the files if they don't already exist:
my_file= Path("opendata-france\\House Prices Paris - Datasources/valuers_foncieres_combined")
#load dataset faster by filtering at source
req_columns=["Year_Price","No disposition", "Date mutation", "Nature mutation", "Valeur fonciere", "Code voie", "Voie", "Code postal", "Commune", "Code departement", "Code commune", "Section", "No plan", "Nombre de lots", "Nature culture", "Surface terrain"]

if my_file.is_file():
    source_file= sources_folder/str("valuers_foncieres_combined")
    #option 1 - faster - use this normally
    #prices = pd.read_csv(source_file,sep= ',',encoding=detect_encoding(source_file), usecols=req_columns)
    #option 2
    prices = pd.read_csv(source_file,sep= ',',encoding=detect_encoding(source_file), nrows=1000)
    
    print("Loaded Combined prices file!")
else:
    
    len(dataSrc2)
    start_year=2015
    i=0

    

    #for each file, download and name it accordingly
    for link in dataSrc2:
        download(link, dest_folder="opendata-france\\House Prices Paris - Datasources", filename="valuers_foncieres_"+str(start_year+i))
        i=i+1

    prices=pd.DataFrame()

    # create master dataset
    for i in range(len(dataSrc2)):
        source_file= sources_folder/str("valuers_foncieres_"+str(start_year+i))
        #load dataset
        df = pd.read_csv(source_file,sep= '|',encoding=detect_encoding(source_file))
        df["Year_Price"]=str(start_year+i)
        prices=prices.append(df)

        # df_2015 = pd.read_csv(sources_folder/str("valuers_foncieres_"+str(2015)),sep= '|')
        # df=df_2015
        # df_2016 = pd.read_csv(sources_folder/str("valuers_foncieres_"+str(2016)),sep= '|')
        # df=df.append(df_2016)
        # df_2017 = pd.read_csv(sources_folder/str("valuers_foncieres_"+str(2017)),sep= '|')
        # df=df.append(df_2017)
        # df_2018 = pd.read_csv(sources_folder/str("valuers_foncieres_"+str(2018)),sep= '|')
        # df=df.append(df_2018)
        # df_2019 = pd.read_csv(sources_folder/str("valuers_foncieres_"+str(2019)),sep= '|')
        # df=df.append(df_2019)
        # df_2020 = pd.read_csv(sources_folder/str("valuers_foncieres_"+str(2020)),sep= '|')
        # df=df.append(df_2020)
    
    # delete unnecessary files
    folder = sources_folder
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        print("Deleting existing files...")
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

    #save dataframe
    prices.to_csv(sources_folder / "valuers_foncieres_combined",index=True)
    print("Saved Combined prices file!")
    

# Create Master Dataset

In [ ]:
# Remove all column which have more than 60% missing values
# ratio = (prices.isna().sum()/len(prices))>0.4
# prices_cleaned=prices[prices.columns[ratio==False]]
prices.head()

In [ ]:
prices_cleaned.head()

# Exploratory Data Analysis

I will start by investigating the distributions of the variables I have on the dataset.  The first step is to identify which variables are categorical and which are numeric. I can then investigate if I need to do any data conversions in between.


In [ ]:
prices_cleaned.info()

In [ ]:
#some data cleaning
prices_cleaned["Valeur fonciere"]=prices_cleaned["Valeur fonciere"].str.replace(',','.').astype(np.float64)
prices_cleaned=prices_cleaned[prices_cleaned["Valeur fonciere"]<100000000]